In [1]:
import numpy as np
import os
import csv
import dask.dataframe as dd

In [2]:
df = dd.read_csv("NIS_2012_CoreCSV.csv", dtype=object)

In [3]:
prcols = []
for i in np.arange(0,15,1):
    prcols.append("PR{}".format(i+1))
prcols.insert(0, "KEY_NIS") # Inserts 'KEY_NIS' as first element
# Create dataframe with only 'KEY_NIS' and procedure columns
dfpr = df[prcols]

In [4]:
dfpr.head()

,KEY_NIS,PR1,PR2,PR3,PR4,PR5,PR6,PR7,PR8,PR9,PR10,PR11,PR12,PR13,PR14,PR15
0,10000011,741,7534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000148,9547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000218,3722,8856,8853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#dfpr.to_csv("ddfpr_rows/*.csv") # Creates csv files from the dask dataframe pieces (34 chunks)

**Procedure Matrix WITH KEY_NIS**

In [6]:
### Procedure Matrix with KEY_NIS
'''
This code reads in the procedure files, creates column (ICD9) and row (KEY_NIS) to index dictionaries, and
obtains the dimensions for the procedure matrix
'''
total_codes = set([]) # len(total_codes)= number columns needed. SET prohibits duplicate values
codes_to_col_index = {} # Dictionary that will store each ICD code as the key and the column number as the value
row_num = 0 # find the total amount of rows
total_ids = set([]) # Array storing unique KEY_NIS identifiers
ids_to_row_index = {} # Dictionary storing each KEY_NIS identifier as the key and the row number as the value


for f in sorted(os.listdir("ddfpr_rows")): # Iterates through each of the 34 files in the dd fpr_rows directory
    read = csv.reader(open("ddfpr_rows/{}".format(f)))
    test = 0
    for row in read:
        if row[0] != '':# if not header row since header rows don't have a first entry (id)
            key_nis = row[1] # The 'KEY_NIS' identifier is located in the second column of each row
            if key_nis not in ids_to_row_index:
                ids_to_row_index[key_nis] = len(total_ids) # Alternatively, ...= row_num
                total_ids.add(key_nis)
            else:
                print("Repeat Visit for Individual: {}".format(key_nis))
            for code in row[2:]: #iterate through each item in the row (excluding fake index at [1] and KEY_NIS at [2])
                if code == '': # if we've reached the end of the values, leave the row
                    break
                else:
#                    print(code)
                    if code not in codes_to_col_index:
                        codes_to_col_index[code] = len(total_codes)
                        # in 'codes_to_index' dictionary, creates key of 'code' & assigns key value of len(total_codes)
                        # to keep track of the cols of the mtx (the ICD9 codes) and their indices
                        total_codes.add(code)
                        # adds the new code to the 'total_codes' set of unique code values
            row_num +=1
            if row_num % 100000 == 0:
                print("HERE: ", row_num)         

HERE:  100000
HERE:  200000
HERE:  300000
HERE:  400000
HERE:  500000
HERE:  600000
HERE:  700000
HERE:  800000
HERE:  900000
HERE:  1000000
HERE:  1100000
HERE:  1200000
HERE:  1300000
HERE:  1400000
HERE:  1500000
HERE:  1600000
HERE:  1700000
HERE:  1800000
HERE:  1900000
HERE:  2000000
HERE:  2100000
HERE:  2200000
HERE:  2300000
HERE:  2400000
HERE:  2500000
HERE:  2600000
HERE:  2700000
HERE:  2800000
HERE:  2900000
HERE:  3000000
HERE:  3100000
HERE:  3200000
HERE:  3300000
HERE:  3400000
HERE:  3500000
HERE:  3600000
HERE:  3700000
HERE:  3800000
HERE:  3900000
HERE:  4000000
HERE:  4100000
HERE:  4200000
HERE:  4300000
HERE:  4400000
HERE:  4500000
HERE:  4600000
HERE:  4700000
HERE:  4800000
HERE:  4900000
HERE:  5000000
HERE:  5100000
HERE:  5200000
HERE:  5300000
HERE:  5400000
HERE:  5500000
HERE:  5600000
HERE:  5700000
HERE:  5800000
HERE:  5900000
HERE:  6000000
HERE:  6100000
HERE:  6200000
HERE:  6300000
HERE:  6400000
HERE:  6500000
HERE:  6600000
HERE:  6700000
HERE

In [7]:
### Builds the Procedure Matrix using the dimensions obtained from the previous code.
testmtx = np.zeros((row_num, len(total_codes)), dtype=np.bool)

row_num = 0

for f in sorted(os.listdir("ddfpr_rows")):
    read = csv.reader(open("ddfpr_rows/{}".format(f)))
    test = 0
    for row in read:
        if row[0] != '': # if not header row
            
#             print(row)
#             print("ROW NUM", row_num)
            for code in row[2:]: #iterate through each item in the row (excluding fake index [1] and KEY_NIS at [2])
                if code == '': # if we've reached the end of the values, leave the row
                    break
                else:
#                    test_codes.add(code)
#                    print(code)
                    try:
                        testmtx[row_num, codes_to_col_index[code]] = True
                    except:
                        pass
#                     print("SUM", np.sum(testmtx[row_num]))
            row_num +=1
            if row_num % 100000 == 0:
                print("HERE: ", row_num)            

HERE:  100000
HERE:  200000
HERE:  300000
HERE:  400000
HERE:  500000
HERE:  600000
HERE:  700000
HERE:  800000
HERE:  900000
HERE:  1000000
HERE:  1100000
HERE:  1200000
HERE:  1300000
HERE:  1400000
HERE:  1500000
HERE:  1600000
HERE:  1700000
HERE:  1800000
HERE:  1900000
HERE:  2000000
HERE:  2100000
HERE:  2200000
HERE:  2300000
HERE:  2400000
HERE:  2500000
HERE:  2600000
HERE:  2700000
HERE:  2800000
HERE:  2900000
HERE:  3000000
HERE:  3100000
HERE:  3200000
HERE:  3300000
HERE:  3400000
HERE:  3500000
HERE:  3600000
HERE:  3700000
HERE:  3800000
HERE:  3900000
HERE:  4000000
HERE:  4100000
HERE:  4200000
HERE:  4300000
HERE:  4400000
HERE:  4500000
HERE:  4600000
HERE:  4700000
HERE:  4800000
HERE:  4900000
HERE:  5000000
HERE:  5100000
HERE:  5200000
HERE:  5300000
HERE:  5400000
HERE:  5500000
HERE:  5600000
HERE:  5700000
HERE:  5800000
HERE:  5900000
HERE:  6000000
HERE:  6100000
HERE:  6200000
HERE:  6300000
HERE:  6400000
HERE:  6500000
HERE:  6600000
HERE:  6700000
HERE

In [25]:
print(len(total_codes))
print(row_num)

3621
7296968


In [30]:
rindex_to_ids = {ids_to_row_index[k] : k for k in ids_to_row_index}
cindex_to_codes = {codes_to_col_index[k] : k for k in codes_to_col_index}

In [73]:
# Verify that the KEY_NIS identifier matches number of procedures and procedures codes
for i in np.arange(0,5,1):
    temp = []
    for index, item in enumerate(testmtx[i]):
        if item == True:
            #print(cindex_to_codes[index])
            temp.append(cindex_to_codes[index])
        else:
            pass
    print("KEY_NIS: {}\tNum of Prcds: {}\t    Procedure Codes: {}".format(rindex_to_ids[i], np.sum(testmtx[i,:]),
                                                                          [x for x in temp]))

KEY_NIS: 10000011	Num of Prcds: 2	    Procedure Codes: ['741', '7534']
KEY_NIS: 10000148	Num of Prcds: 1	    Procedure Codes: ['9547']
KEY_NIS: 10000174	Num of Prcds: 0	    Procedure Codes: []
KEY_NIS: 10000218	Num of Prcds: 3	    Procedure Codes: ['3722', '8856', '8853']
KEY_NIS: 10000229	Num of Prcds: 0	    Procedure Codes: []


In [28]:
# Generate list of procedure columns
prcols = []
for i in np.arange(0,15,1):
    prcols.append("PR{}".format(i+1))
prcols.insert(0, "KEY_NIS")
dfpx = df[prcols] # create dataframe of only procedure columns and 'KEY_NIS', which is a unique VISIT identifier
dfpx.head()

,KEY_NIS,PR1,PR2,PR3,PR4,PR5,PR6,PR7,PR8,PR9,PR10,PR11,PR12,PR13,PR14,PR15
0,10000011,741,7534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000148,9547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000218,3722,8856,8853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
testmtx.shape

(7296968, 3621)

**Procedure Matrix WITHOUT KEY_NIS**

In [ ]:
# testmtx = np.zeros((7296968, 10000), dtype=np.bool)
# add all codes in here
total_codes = set([]) # len(total_codes)= number columns needed. SET prohibits duplicate values
codes_to_index = {} # store code as key, and col_num as value
row_num = 0 # find the total amount of rows

for f in sorted(os.listdir("dfpr_rows")): # Iterates through each of the 34 files in the dd fpr_rows directory
    read = csv.reader(open("dfpr_rows/{}".format(f)))
    test = 0
    for row in read:
        if row[0] != '': # if not header row since header rows don't have a first entry (id)
#             print(row)
#             print("ROW NUM", row_num)
            for code in row[1:]: #iterate through each item in the row (excluding fake index)
                if code == '': # if we've reached the end of the values, leave the row
                    break
                else:
#                     print(code)
                    if code not in codes_to_index:
                        codes_to_index[code] = len(total_codes)
                        # in 'codes_to_index' dictionary, creates key of 'code' & assigns key value of len(total_codes)
                        # to keep track of the cols of the mtx (the ICD9 codes) and their indices
                        total_codes.add(code)
                        # adds the new code to the 'total_codes' set of unique code values
                    try:
                        testmtx[row_num, int(code)] = True
                    except:
                        pass
#                     print("SUM", np.sum(testmtx[row_num]))
            row_num +=1
            if row_num % 100000 == 0:
                print("HERE: ", row_num)         

In [ ]:
testmtx = np.zeros((row_num, len(total_codes)), dtype=np.bool)
# total_codes = set([])
row_num = 0
# test_codes.add(code)
for f in sorted(os.listdir("dfpr_rows")):
    read = csv.reader(open("dfpr_rows/{}".format(f)))
    test = 0
    for row in read:
        if row[0] != '': # if not header row
#             print(row)
#             print("ROW NUM", row_num)
            for code in row[1:]: #iterate through each item in the row (excluding fake index)
                if code == '': # if we've reached the end of the values, leave the row
                    break
                else:
#                 test_codes.add(code)
#                     print(code)
                    try:
                        testmtx[row_num, int(code)] = True
                    except:
                        pass
#                     print("SUM", np.sum(testmtx[row_num]))
            row_num +=1
            if row_num % 100000 == 0:
                print("HERE: ", row_num)
            